In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install bertopic

In [3]:
import time
import pandas as pd
import json
import matplotlib.pyplot as plt
import requests
import os
from tqdm import tqdm
from bertopic import BERTopic
import numpy as np

In [18]:
from sklearn.metrics.pairwise import euclidean_distances

In [205]:
def data_processing(origin_df):
    # 원본 데이터 전처리
    origin_df.drop(['Unnamed: 0'],axis=1,inplace=True)
    origin_df = origin_df.drop_duplicates()
    origin_df = origin_df.dropna()
    origin_df.reset_index(inplace=True,drop=True)

    return origin_df

In [209]:
def bertopic_distance(df,origin_df,path):
    #### 1. 골든크로스 데이터 BerTopic 적합
    documents = df['title']
    topic_model = BERTopic(embedding_model=None, verbose=True)
    golden_embedding_df = df.iloc[:,6:].to_numpy()
    topics, probabilities = topic_model.fit_transform(documents, golden_embedding_df)

    #### 2. 토픽별로 중심점으로부터 거리 계산 후 중심점으로부터 가까운 순서대로 정렬
    df['topic'] = topics #BerTopic 결과 저장
    df = df[df['topic'] != -1] # Outlier 제거
    df['distance_to_center'] = np.nan  # 새로운 컬럼 추가
    df['topic_name'] = np.nan  # 새로운 컬럼 추가
    topic_name_list = topic_model.generate_topic_labels(nr_words=4)[1:]

    # 각 토픽별로 처리
    for topic in sorted(list(df['topic'].unique())):
        # 특정 토픽에 속하는 데이터 프레임
        topic_df = df[df['topic'] == topic]
        # 필요한 열만 추출
        topic_embedding_df = topic_df.iloc[:, 6:-3]
        # 행렬로 변환
        topic_matrix = topic_embedding_df.to_numpy()
        # 중심 벡터
        center_vector = topic_model.topic_embeddings_[topic].reshape(1, -1)
        # 유클리드 거리 계산
        distances = euclidean_distances(topic_matrix, center_vector).flatten()
        # 거리 값을 원본 데이터프레임에 추가
        df.loc[df['topic'] == topic, 'distance_to_center'] = distances
        df.loc[df['topic'] == topic, 'topic_name'] = topic_name_list[topic]

    # 정렬된 데이터프레임 생성 (거리 기준으로)
    golden_df_center = df.sort_values(by=['topic', 'distance_to_center'])

    #### 3. 임베딩값들을 지우고, 컬럼 정렬
    final_df = golden_df_center.iloc[:,:6]
    final_df['topic'] =  golden_df_center['topic']
    final_df['distance_to_center'] = golden_df_center['distance_to_center']
    final_df['topic_name'] = golden_df_center['topic_name']
    final_df['origin_title'] = origin_df.loc[final_df.index,:]['title']

    final_df = final_df[['topic','topic_name','distance_to_center','origin_title','content','link','date','TargetDate','CrossType','title']]
    final_df.reset_index(drop=True,inplace=True)

    final_df.to_excel(path,index=False)

    return final_df, topic_model

In [210]:
def bertopic_summary(final_df,topic_model):
    #### 4. Topic별 count, topic_name, 대표 단어, 대표 title
    topic_summary = pd.DataFrame()
    for idx, topic in enumerate(sorted(list(final_df['topic'].unique()))):
        topic_df = final_df[final_df['topic']==topic]
        topic_summary.loc[idx,'topic'] = int(topic)
        topic_summary.loc[idx,'topic_name'] = topic_model.generate_topic_labels(nr_words=4)[1:][topic]
        topic_summary.loc[idx,'count'] = int(len(topic_df))
        topic_summary.loc[idx,'Representation_title'] = str(list(topic_df['origin_title'])[:3])

    return topic_summary

In [211]:
def topic_per_10(final_df,path):
    #### 5. 요약을 위해서, 각 토픽별로 거리가 가까운 순서대로 10개의 기사를 추출할 예정임
    # 5-1. 각 토픽 내 뉴스 기사의 개수가 전체 토픽의 뉴스 기사 개수의 중위수보다 클 경우의 토픽만 채택
    # 각 토픽별 개수 계산
    topic_counts = final_df['topic'].value_counts()
    # 토픽별 개수의 중위수 계산
    median_count = topic_counts.median()
    # 중위수보다 높은 토픽들 추출
    median_topic = topic_counts[topic_counts>median_count].index

    # 5-2. 각 토픽별로 거리가 가까운 순서대로 10개의 기사를 추출할 예정임
    topic_per_10 = pd.DataFrame()
    for topic in median_topic:
        topic_df = final_df[final_df['topic']==topic]
        topic_df.sort_values(by=['distance_to_center'],inplace=True)
        topic_per_10 = pd.concat([topic_per_10,topic_df.iloc[:10,:]])

    topic_per_10.to_excel(path,index=False)

    return topic_per_10

In [212]:
# 골든크로스 데이터프레임, 원본 데이터(원본 제목 용도) 가져오기
golden_df = pd.read_csv('/content/drive/MyDrive/Miraeasset/data//embedded_news_df_삼성전자_golden_2023-02-16.csv')
golden_origin_title = pd.read_csv('/content/drive/MyDrive/Miraeasset/data/news_df_삼성전자_golden_2023-02-16.csv')
golden_origin_title = data_processing(golden_origin_title)

# 반도체 섹터 데이터프레임 가져오기
news_df = pd.read_csv('/content/drive/MyDrive/Miraeasset/data/embedded_news_df_반도체_2023-02-16.csv')

In [213]:
path = '/content/drive/MyDrive/Miraeasset/data/embedded_news_df_삼성전자_golden_2023-02-16_BERTopic.xlsx'
top10_path = '/content/drive/MyDrive/Miraeasset/data/embedded_news_df_삼성전자_golden_2023-02-16_BERTopic_10.xlsx'
final_df, topic_model = bertopic_distance(golden_df, golden_origin_title,path)
topic_summary = bertopic_summary(final_df,topic_model)
topic_per_10 = topic_per_10(final_df,top10_path)

2024-07-27 08:39:27,430 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-07-27 08:39:34,269 - BERTopic - Dimensionality - Completed ✓
2024-07-27 08:39:34,272 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-07-27 08:39:34,322 - BERTopic - Cluster - Completed ✓
2024-07-27 08:39:34,328 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-07-27 08:39:34,369 - BERTopic - Representation - Completed ✓
